# **Data analisys for validation MODIS vs VESPA-22 IWV measurements**

## **Import libraries**

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices

from scipy import stats

## **Selected data reception from Thule-mix-LAADS.ipynb**

In [ ]:
%store -r max_IWV
%store -r max_dist_from_THAAO
%store -r deltatime
%store -r new_mod_vespa
%store -r new_myd_vespa
%store -r new_mxd_vespa
# passaggio plot_defaults
%store -r plot_defaults
# passaggio dati completi non filtrati con deltatime
%store -r vespa_data

### **Controllo dati ricevuti**

In [ ]:
vespa_data.head(1)     # ok: raw per test


In [ ]:
# converto da datetime a timestamp (float in seconds, default=ns)
vespa_data['timestamp'] = pd.to_numeric(vespa_data['Time'].values)/ 10 ** 9  # to timestamp in seconds (default=ns)
type(vespa_data['timestamp'][2])

In [ ]:
# ok: raw per test
print(type(vespa_data['Time'][0]))
vespa_data.head(1)    

In [ ]:
num_mod_data_p_Dt = len(new_mod_vespa.IWV_MODIS) 
num_myd_data_p_Dt = len(new_myd_vespa.IWV_MODIS)
num_mxd_data_p_Dt = len(new_mxd_vespa.IWV_MODIS)
num_modmyd_data_p_Dt = num_mod_data_p_Dt + num_myd_data_p_Dt

In [ ]:
print(f'-- Selection parameters --')
print(f'max IWV = {max_IWV} [mm]')
print(f'max distance from THAOO = {max_dist_from_THAAO} [km]')
print(f'Deltatime chosen for data points selection = {deltatime}\n')
print(f'-- Data points selected --')
print(f'Selected number of MOD data points =          {num_mod_data_p_Dt}')
print(f'Selected number of MYD data points =          {num_myd_data_p_Dt}\n')
print(f'Total selected number of MXD data points     =  {num_mxd_data_p_Dt}')
print(f'Total selected number of MOD+MYD data points =  {num_modmyd_data_p_Dt}\n')

## **Analisi spettrale**

### **Selezione e plot dei dati in un intervallo temporale** 

In [ ]:
import hvplot.pandas

color_THAAO = 'green'

In [ ]:
# seleziono un intervallo temporale che voglio analizzare 
# NOTA: .copy() è necessario per evitare una shallow copy ed il settingwithcopywarning
vespa_time_range = vespa_data.loc[vespa_data["Time"].between('2019-01-01 00:00:00','2021-01-01 00:00:00'),:].copy()

In [ ]:
# test selezione
print(len(vespa_time_range))
vespa_time_range.head(3)  #.tail(3)

In [ ]:
vespa_ts_scatter = vespa_time_range.IWV_THAAO.hvplot.scatter(color=color_THAAO, label='VESPA',
                                                       fields={'date': 'Date'},
                                                       title='IWV THAAO - VESPA', 
                                                       **plot_defaults)

In [ ]:
#(vespa_ts_scatter*vespa_ts_scatter_rec).opts(show_grid=True)
(vespa_ts_scatter).opts(show_grid=True)

### **Interpolazione**
Poiché i dati non sono equispaziati e contengono buchi provo ad interpolare prima di fare lo spettro <br>

vedi [gaps](http://mres.uni-potsdam.de/index.php/2017/08/22/data-voids-and-spectral-analysis-dont-be-afraid-of-gaps/) <br>
**NOTA**: Since all these (and other) interpolation techniques might introduce artifacts into the data, <br> 
it is always advisable to (1) keep the total number of data points constant before and after <br>
interpolation, (2) report the method employed for estimating the evenly-spaced data sequence, <br>
and (3) explore the effect of interpolation on the variance of the data.

In [ ]:
# creo la time series di timestamp con dati temporalmente equispaziati

In [ ]:
print(vespa_time_range['Time'][0])
print(vespa_time_range['Time'].iloc[-1])
print(len(vespa_time_range['Time']))

In [ ]:
date_rng = pd.date_range(start=vespa_time_range['Time'][0], 
                         end=vespa_time_range['Time'].iloc[-1], 
                         periods=len(vespa_time_range['Time']))
print(len(date_rng))
print(date_rng[0])
print(date_rng[-1])
print(type(date_rng))

In [ ]:
vespa_time_range['new_date_rng'] = date_rng

In [ ]:
vespa_time_range.tail(2)

In [ ]:
# converto il time range selezionato da dateime a timestamp (float in seconds, default=ns)
vespa_time_range['new_timestamp_rng'] = pd.to_numeric(vespa_time_range['new_date_rng'].values)/ 10 ** 9  # to timestamp in seconds (default=ns)
vespa_time_range.tail(2)

In [ ]:
vespa_time_range = vespa_time_range.set_index(['Time','new_date_rng'], 
                            drop=False).sort_index()

#### **Linear, Cubic spline interpolation, Piecewise Cubic Hermite Interpolating Polynomial interpolation, Akima1D**

In [ ]:
from scipy.interpolate import interp1d, CubicSpline, PchipInterpolator, Akima1DInterpolator

In [ ]:
linear = interp1d(vespa_time_range['timestamp'], vespa_time_range['IWV_THAAO'], kind='linear')

Interpolo sui dati reali con le date datetime convertite in timestamp

In [ ]:
CS = CubicSpline(vespa_time_range['timestamp'],
                 vespa_time_range['IWV_THAAO'])

In [ ]:
PCHIP = PchipInterpolator(vespa_time_range['timestamp'], 
                          vespa_time_range['IWV_THAAO'], 
                          axis=0, extrapolate=None)

In [ ]:
Akima1D = Akima1DInterpolator(vespa_time_range['timestamp'],
                              vespa_time_range['IWV_THAAO'])

In [ ]:
vespa_time_range['linear'] =  linear(vespa_time_range['new_timestamp_rng'])
vespa_time_range['CS'] =  CS(vespa_time_range['new_timestamp_rng'])
vespa_time_range['PCHIP'] =  PCHIP(vespa_time_range['new_timestamp_rng'])
vespa_time_range['Akima1D'] =  Akima1D(vespa_time_range['new_timestamp_rng'])

In [ ]:
vespa_time_range.loc[vespa_time_range['Time']=='2020-01-08 15:56:28']

In [ ]:
vespa_time_range['new_timestamp_rng'][1]-vespa_time_range['new_timestamp_rng'][0]

In [ ]:
type(vespa_time_range.new_date_rng[0])
#type(vespa_time_range.Time[0])

In [ ]:
vespa_linear = vespa_time_range.linear.hvplot.scatter(x='new_date_rng',
                                                      color='blue',
                                                      label='linear',
                                                      title='IWV THAAO - VESPA',
                                                      **plot_defaults)
vespa_CS = vespa_time_range.CS.hvplot.scatter(x='new_date_rng',
                                              color='red',
                                              label='CS',
                                              title='IWV THAAO - VESPA',
                                              **plot_defaults)
vespa_PCHIP = vespa_time_range.PCHIP.hvplot.scatter(x='new_date_rng', 
                                                    color='cyan', 
                                                    label='PCHIP',
                                                    title='IWV THAAO - VESPA', 
                                                    **plot_defaults)
vespa_Akima1D = vespa_time_range.Akima1D.hvplot.scatter(x='new_date_rng', 
                                                        color='yellow', 
                                                        label='Akima1D',
                                                        title='IWV THAAO - VESPA', 
                                                        **plot_defaults)

#### **Plot of interpolations and original data**

In [ ]:
# grafico vespa nel range temporale definito ed interpolazione
(vespa_ts_scatter
*vespa_linear
#*vespa_CS
*vespa_PCHIP
#*vespa_Akima1D
)

### **Periodogrammi**

#### **Dati equispaziati (interpolati)**

In [ ]:
# Passo di campionamento dt (sample spacing, secondi tra un dato ricampionato e il successivo)
dt = vespa_time_range['new_timestamp_rng'][10002]-vespa_time_range['new_timestamp_rng'][10001]

##### **FFT**

In [ ]:
# Fourier Analysis in Python
from scipy.fft import fft, fftfreq
import numpy as np

# Number of sample points
N = len(vespa_time_range)

x = (vespa_time_range['new_timestamp_rng']).values #np.linspace(0.0, N*T, N, endpoint=False)
y = (vespa_time_range['PCHIP']).values #np.sin(50.0 * 2.0*np.pi*x) + 0.5*np.sin(80.0 * 2.0*np.pi*x)
yf = fft(y)
xf = fftfreq(N, dt)[0:N//2]

print(len(xf), len(2.0/N * np.abs(yf[0:N//2])), len(yf), ' N =',N)

In [ ]:
# Nota unità di misura: xf_s [s**(-1)], xf_d [days**(-1)] 
#print(xf[1], 2.0/N * np.abs(yf[1//2]))
print(len(xf), len(2.0/N * np.abs(yf[0:N//2])),  ' N = ',N )
print(type(xf), type(2.0/N * np.abs(yf[0:N//2])))
FFT_all_f = pd.DataFrame({'xf_s':xf, 'xf_d':xf*86400, 'yf':2.0/N * np.abs(yf[0:N//2])}) 
#FFT_all_f.set_index('xf_s',drop=True,inplace=True)
FFT_all_f.head()

In [ ]:
# drop first row to plot log(x)
FFT = FFT_all_f.iloc[1:].copy()
print(len(FFT))
FFT['xt_d'] = (1./FFT.xf_d)  # unità di misura: xt_d [days] 
FFT.set_index(['xf_s','xf_d'],drop=True,inplace=True)
f_max_gg = FFT.yf*86400
FFT.head(2)

In [ ]:
import hvplot.pandas
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool,WheelZoomTool, PanTool, ResetTool
#hvplot.extension('bokeh')

In [ ]:
# tooltips cambia solo il nome nel hovertool
hover = HoverTool(tooltips=[('f_seconds','@xf_s'),('t_days','@xt_d'),('FFT','@yf')])
FFT.hvplot.line(x='xf_s', y='yf', logx=True, grid=True, hover_cols=['xf_s','yf','xt_d']).opts(tools=[hover])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print(1/dt)
f_max = 3*(10**-8)     # Hz
f_max_gg = f_max*86400      # 1/giorni
f_max_2 = 6.1*(10**-8)     # Hz
f_max_gg_2 = f_max_2*86400      # 1/giorni
print(f'f_max={f_max} [Hz], f_max_gg={f_max} [day-1]')
TT=(1./(f_max_gg))
print(f'TT = {TT} [days]')

print(f'f_max_2={f_max_2} [Hz], f_max_gg_2={f_max_2} [day-1]')
TT_2=(1./(f_max_gg_2))
print(f'TT_2 = {TT_2} [days]')
#TT/86400 # a okkio 579 giorni il picco più grande

##### **PS and PSD** 
Power Spectrum e Power Spectral Density base <br> 
(scipy.signal.periodogram(data,fs,scaling='spectrum') <br>
unità di misura: <br>
IWV: $[mm^2]$ <br>
frequency: $[Hz]$

In [ ]:
from scipy import signal
fs = 1/dt 
T = len(vespa_time_range)*dt  # T=N*dt [s]
df = 1/T  # df=1/T=fs/N , dim bin in freq. df = (1/(N*dt))[Hz]=fatt. di conv

In [ ]:
f_ps, PS_f = signal.periodogram(vespa_time_range.PCHIP,fs, 
                                window='blackman', #'boxcar', #'hann', 
                                scaling='spectrum')

In [ ]:
f_psd, PSD_f = signal.periodogram(vespa_time_range.PCHIP,fs, scaling='density')

In [ ]:
# print di controllo
print(f'len(vespa_time_range) = {len(vespa_time_range)}')
print(f'len(PS)  = {len(PS_f)}, len(f_ps)  = {len(f_ps)}')
print(f'len(PSD) = {len(PSD_f)}, len(f_psd) = {len(f_ps)}\n')

print(f'dt = {dt} [s], fs = {fs} [s^-1]')
print(f'dt_o = {dt/3600} [hours]')
print(f'T   = {T} [s], df = {df} 1/(N*dt))[Hz]')
print(f'T_a = {T/(365*86400)} [years]\n')

print(f'min(PS) = {min(PS_f)}, max(PS) = {max(PS_f)}')
print(f'min(PSD) = {min(PSD_f)}, max(PSD) = {max(PSD_f)}')
print(f'min(f_ps)  = {min(f_ps)}, max(f_ps)  = {max(f_ps)}')
print(f'min(f_psd) = {min(f_psd)}, max(f_psd) = {max(f_psd)}\n')

In [ ]:
PS_PSD_f = pd.DataFrame({'f_ps':f_ps, 'f_ps_d':f_ps*86400, 'PS':PS_f,
                        'f_psd':f_psd, 'f_psd_d':f_psd*86400, 'PSD':PSD_f}) #2.0/N * np.abs(yf[0:N//2])}) 
PS_PSD_f.head(2)

In [ ]:
# drop first row to plot log(x)
PS_PSD = PS_PSD_f.iloc[1:].copy()
print(len(f_ps))
PS_PSD['t_ps_d'] = (1./PS_PSD.f_ps_d)  # unità di misura: xt_d [days] 
PS_PSD['t_psd_d'] = (1./PS_PSD.f_psd_d)  # unità di misura: xt_d [days] 
#PS_PSD.set_index(['f_ps','f_ps_d','f_psd','f_psd_d'],drop=True,inplace=True)
PS_PSD.set_index(['f_ps','f_psd'],drop=True,inplace=True)
PS_PSD.head(2)

In [ ]:
# tooltips cambia il nome nel hovertool
hover = HoverTool(tooltips=[('f_ps','@f_ps'),('t_ps_d','@t_ps_d'),('PS','@PS')])

In [ ]:
PS_PSD.hvplot.line(x='f_ps', y='PS', logx=True, logy=True, grid=True, 
                   xlabel='frequency [Hz]', ylabel='PS [mm**2]', 
                   hover_cols=['f_ps','PS','t_ps_d']).opts(tools=[hover])

In [ ]:
# tooltips cambia il nome nel hovertool
hover = HoverTool(tooltips=[('f_psd','@f_psd'),('t_psd_d','@t_psd_d'),('PSD','@PSD')])

In [ ]:
PS_PSD.hvplot.line(x='f_psd', y='PSD', logx=True, logy=True, grid=True, 
                   xlabel='frequency [Hz]', ylabel='PSD [mm**2/Hz]', 
                   hover_cols=['f_psd','PSD','t_psd_d']).opts(tools=[hover])

##### **PSD Welch** 
Power Spectral Density Welch <br>
(scipy.signal.welch(data, fs, nperseg=1024, scaling='spectrum') <br>
unità di misura: <br>
IWV: $[mm^2]$ <br>
frequency: $[Hz]$

In [ ]:
f_w, PSD_welch = signal.welch(vespa_time_range.PCHIP, fs,
                              #scaling='spectrum',
#                              window='boxcar', 
#                              window='hann', 
#                              window='blackman', 
                              window=('kaiser', 14), 
                              nperseg=len(vespa_time_range.PCHIP))
len(PSD_welch)

In [ ]:
PSD_wel = pd.DataFrame({'f_w':f_w, 'f_w_d':f_w*86400, 'PSD_welch':PSD_welch}) 
print(PSD_wel.head(2))
# drop first row to plot log(x)
PSD_w = PSD_wel.iloc[1:].copy()
print(len(f_ps))
PSD_w['t_w_d'] = (1./PSD_w.f_w_d)  # unità di misura: xt_d [days] 
PSD_w.set_index(['f_w'],drop=True,inplace=True)
PSD_w.head(2)

In [ ]:
# tooltips cambia il nome nel hovertool
hover = HoverTool(tooltips=[('f_w','@f_w'),('t_w_d','@t_w_d'),('PSD_welch','@PSD_welch')])

In [ ]:
PSD_w.hvplot.line(x='f_w', y='PSD_welch', logx=True, logy=True, grid=True, 
                   xlabel='frequency [Hz]', ylabel='PSD Welch [mm**2/Hz]', 
                   hover_cols=['f_w','PSD_welch','t_w_d']).opts(tools=[hover])

In [ ]:
#Pxx_den.hvplot.line(x='f', y='Pxx_den')

#### **Error/Trend/Seasonality (ETS) model**

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np


In [ ]:
new_mxd_vespa_1 = new_mxd_vespa.reset_index(drop=True)

In [ ]:
new_mxd_vespa_1 = new_mxd_vespa_1.set_index('Time')
new_mxd_vespa_1 = new_mxd_vespa_1.sort_index()

In [ ]:
#new_mxd_vespa_1 = new_mxd_vespa_1.asfreq(pd.infer_freq(new_mxd_vespa_1.index),method='bfill')

In [ ]:
#new_mxd_vespa_1.asfreq(method='bfill')

In [ ]:
len(vespa_time_range.PCHIP)
vespa_time_range.head(2)

In [ ]:
# print di controllo
print(f'len(vespa_time_range) = {len(vespa_time_range)}')
print(f'len(PS)  = {len(PS_f)}, len(f_ps)  = {len(f_ps)}')
print(f'len(PSD) = {len(PSD_f)}, len(f_psd) = {len(f_ps)}\n')

print(f'dt = {dt} [s], fs = {fs} [s^-1]')
print(f'dt_o = {dt/3600} [hours]')
print(f'T   = {T} [s], df = {df} 1/(N*dt))[Hz]')
print(f'T_a = {T/(365*86400)} [years]\n')

print(f'min(PS) = {min(PS_f)}, max(PS) = {max(PS_f)}')
print(f'min(PSD) = {min(PSD_f)}, max(PSD) = {max(PSD_f)}')
print(f'min(f_ps)  = {min(f_ps)}, max(f_ps)  = {max(f_ps)}')
print(f'min(f_psd) = {min(f_psd)}, max(f_psd) = {max(f_psd)}\n')

In [ ]:
# ricreo un df pandas con i dataset che mi interessano 
vespa_ETS_timerange = (vespa_time_range[['new_date_rng', 'new_timestamp_rng','PCHIP']].copy()).droplevel(0).drop('new_date_rng', axis=1)
#vespa_ETS_tr.index[100:200].mean()
vespa_ETS_timerange.head(2)

In [ ]:
T_delta = vespa_ETS_timerange.new_timestamp_rng[101]-vespa_ETS_timerange.new_timestamp_rng[100]
print(T_delta)
periodo = int(365*86400/T_delta)
periodo 

In [ ]:
result_add = seasonal_decompose(vespa_ETS_timerange.PCHIP,
                                #period=T_delta.seconds,
                                period=periodo,
                                model = 'add', extrapolate_trend='freq')

In [ ]:
#print(f'trend={result_add.trend},\n\n seasonal={result_add.seasonal},\n\n resid={result_add.resid}')

In [ ]:
result_add.plot();

In [ ]:
from statsmodels.tsa.seasonal import STL

stl = STL(vespa_ETS_timerange.PCHIP, period=periodo, seasonal=13)
res = stl.fit()
fig = res.plot()

In [ ]:
print(vespa_ETS_timerange.index.freq)

#### **Lomb-Scargle periodogram for unevenly spaced data**

In [ ]:
from astropy.timeseries import LombScargle
import astropy.units as u

dy = 0.05          # 5% error on VESPA measurements

In [ ]:
frequency, power = LombScargle(vespa_time_range['timestamp'],
                               vespa_time_range['IWV_THAAO'], 
                               dy#, 
                               # normalization='psd'
                              ).autopower()
minimum_frequency = min(frequency)
print(f'min(frequency) = {min(frequency)}, max(frequency) = {max(frequency)}')
print(f'min(power) = {min(power)}, max(power) = {max(power)}')
print(f'min(vespa_time_range.IWV_THAAO) = {min(vespa_time_range.IWV_THAAO.values)}')
print(f'max(vespa_time_range.IWV_THAAO) = {max(vespa_time_range.IWV_THAAO.values)}')
print(f'min(vespa_time_range.timestamp) = {min(vespa_time_range.timestamp.values)}')
print(f'max(vespa_time_range.timestamp) = {max(vespa_time_range.timestamp.values)}\n')
print(f'len(vespa_time_range) = {len(vespa_time_range)}')
print(f'len(power) = {len(power)}')
print(f'len(frequency) = {len(frequency)}\n')

In [ ]:
vespa_data['IWV_THAAO'].isnull().values.any()

In [ ]:
vespa_dropna = vespa_data.dropna(subset = ['IWV_THAAO'])

In [ ]:
vespa_dropna['IWV_THAAO'].isnull().values.any()

In [ ]:
frequency, power = LombScargle(vespa_dropna['timestamp'], 
                               vespa_dropna['IWV_THAAO'], 
                               dy#,
                               #fit_mean=True#,
                               # normalization='psd'
                              ).autopower() #nyquist_factor=10)#) #method='fast')

minimum_frequency = min(frequency)
maximum_frequency = max(frequency)
print(f'min(frequency) = {min(frequency)}, max(frequency) = {max(frequency)}')
print(f'min(power) = {min(power)}, max(power) = {max(power)}')
print(f'min(vespa_dropna.IWV_THAAO) = {min(vespa_dropna.IWV_THAAO.values)}')
print(f'max(vespa_dropna.IWV_THAAO) = {max(vespa_dropna.IWV_THAAO.values)}')
print(f'min(vespa_dropna.timestamp) = {min(vespa_dropna.timestamp.values)}')
print(f'max(vespa_dropna.timestamp) = {max(vespa_dropna.timestamp.values)}\n')
print(f'len(vespa_dropna) = {len(vespa_dropna)}')
print(f'len(power) = {len(power)}')
print(f'len(frequency) = {len(frequency)}\n')

#maximum_frequency = 2.0e-6
frequency, power = LombScargle(vespa_dropna['timestamp'], 
                               vespa_dropna['IWV_THAAO'], 
                               dy#,
                               #fit_mean=True#,
                               # normalization='psd'
                              ).autopower(minimum_frequency=minimum_frequency,
                                          maximum_frequency=maximum_frequency,
                                          samples_per_peak=10
                                         ) 

#minimum_frequency = min(frequency)
print(f'min(frequency) = {min(frequency)}, max(frequency) = {max(frequency)}')
print(f'min(power) = {min(power)}, max(power) = {max(power)}')
print(f'min(vespa_dropna.IWV_THAAO) = {min(vespa_dropna.IWV_THAAO.values)}')
print(f'max(vespa_dropna.IWV_THAAO) = {max(vespa_dropna.IWV_THAAO.values)}')
print(f'min(vespa_dropna.timestamp) = {min(vespa_dropna.timestamp.values)}')
print(f'max(vespa_dropna.timestamp) = {max(vespa_dropna.timestamp.values)}\n')
print(f'len(vespa_dropna) = {len(vespa_dropna)}')
print(f'len(power) = {len(power)}')
print(f'len(frequency) = {len(frequency)}\n')

In [ ]:
PSD_LS = pd.DataFrame({'f_LS':frequency, 'f_LS_d':frequency*86400, 'PSD_LS':power}) 
PSD_LS.head(2)

In [ ]:
# print di controllo
print(f'len(vespa_dropna) = {len(vespa_dropna)}')
print(f'len(vespa_time_range) = {len(vespa_time_range)}')
print(f'len(frequency) = {len(frequency)}, len(power) = {len(power)}')

#print(f'len(PSD_LS) = {len(PSD_LS['PSD_LS'])}')#, len(f_LS) = {len(PSD_LSf_LS)}\n')
#print(f'dt = {dt} [s], fs = {fs} [s^-1]')
#print(f'dt_o = {dt/3600} [hours]')
#print(f'T   = {T} [s], df = {df} 1/(N*dt))[Hz]')
#print(f'T_a = {T/(365*86400)} [years]\n')
#print(f'min(f_LS) = {min(f_LS)}, max(PSD_LS) = {max(PSD_LS)}')

In [ ]:
## drop first row to plot log(x)
#LS_PSD_f = LS_PSD_f.iloc[1:].copy()
print(len(PSD_LS['f_LS']))
PSD_LS['t_LS_d'] = (1./PSD_LS.f_LS_d)  # unità di misura: xt_d [days] 
PSD_LS.set_index(['f_LS','f_LS_d'],drop=True,inplace=True)
PSD_LS.head(2)

In [ ]:
# tooltips cambia il nome nel hovertool
hover = HoverTool(tooltips=[('f_LS','@f_LS'),('t_LS_d','@t_LS_d'),('PSD_LS','@PSD_LS')])

In [ ]:
PSD_LS.hvplot.line(x='f_LS', y='PSD_LS', logx=True, logy=True, grid=True, 
                   xlabel='frequency [Hz]', ylabel='PSD LS [mm**2]', 
                   hover_cols=['f_LS','PSD_LS','t_LS_d']).opts(tools=[hover])

In [ ]:
from astropy.timeseries import LombScargle
import astropy.units as u

dy = 0.05

frequency, power = LombScargle(vespa_time_range['timestamp'], vespa_time_range['IWV_THAAO'], 
                               dy#, 
                               # normalization='psd'
                              ).autopower()
print(len(frequency), frequency.min(), frequency.max())
plt.semilogy(frequency, power)
plt.loglog(frequency, power)

## **Statistica**

### **Scipy stats**

#### **Pearson and Spearman correlation coefficients**

In [ ]:
res_pearson = stats.pearsonr(new_mxd_vespa.IWV_MODIS,new_mxd_vespa.IWV_THAAO)

In [ ]:
print(res_pearson)
print(res_pearson.confidence_interval())

**test**: Pearson correlation coefficient's pvalue=0.0 is less than the minimum floating value, see ref.: 
[pvalue](https://https://stackoverflow.com/questions/45914221/minimal-p-value-for-scipy-stats-pearsonr) <br>

In [ ]:
from scipy.stats import beta
from scipy.special import btdtr
ab = 0.5*num_mxd_data_p_Dt
prob = btdtr(ab, ab, 0.5*(1-abs(res_pearson.statistic)))
prob = beta(ab, ab).cdf(0.5*(1-abs(res_pearson.statistic)))
prob

In [ ]:
res_spearman = stats.spearmanr(new_mxd_vespa.IWV_MODIS,new_mxd_vespa.IWV_THAAO)

In [ ]:
print(f'Spearman: statistic={res_spearman.statistic}, pvalue={res_spearman.pvalue}')

### **Statsmodels**

#### **Ordinary Least Square regression (OLS)**

In [ ]:
# Ordinary Least Square regression
mod = smf.ols(formula='IWV_MODIS ~ IWV_THAAO + diff_int + diff_distance + vza', data=new_mxd_vespa)
res = mod.fit()
print(res.summary())

In [ ]:
# Ordinary Least Square regression
mod = smf.ols(formula='IWV_MODIS ~ IWV_THAAO', data=new_mxd_vespa)
res = mod.fit()
print(res.summary())

In [ ]:
res.params

In [ ]:
res.rsquared

In [ ]:
sm.graphics.plot_partregress('IWV_MODIS','IWV_THAAO', ['diff_distance'], data=new_mxd_vespa, obs_labels=False)

# **Prove**